Phương pháp giải

In [1]:
from pprint import pprint
import random

def gcd_euclid(big: int, small: int):
    if big < small:
        return gcd_euclid(small, big)
    a = big
    b = small
    while (b > 0):
        q = a // b
        r = a % b
        if r == 0:
            break
        a = b
        b = r 
    return b

def is_prime(n: int):
    if n > 360_000_000_000:
        return is_prime_miller_rabin(n)
    return is_prime_naive(n)

def is_prime_naive(n):
    if (n == 1):
        return False
    if (n == 2 or n == 3):
        return True
    if (n % 2 == 0 or n % 3 == 0):
        return False
    # A prime that is not 2 or 3
    # must be in a form of 6k + 1 or 6k + 5
    d = 5
    while(d * d <= n):
        if (n % d == 0 or n % (d + 2) == 0): # check for 6k - 1 aka 6k + 5 and 6k + 1
            return False
        d += 6 # skipping 6k + 2/3/4 (6k + 5 is actually 6k - 1, and we start at 5 aka 6 * 1 - 1)
    return True

def is_prime_miller_rabin(n, rounds:int=500):
    # Miller-Rabin can ensure if a number is not a prime (if False => confidence = 100%),
    # but cannot ensure if a number is prime (if True => confidence != 100%)
    if n % 2 == 0:
        return False
    # Factor two out
    temp = n -1
    s = 0
    while (temp > 1 and temp % 2 == 0):
        temp = temp // 2
        s += 1
    d = temp
    # print(f"{n-1=} {d=} {s=}" )
    # Hybrid approrach with some deterministic bases for number < 2^64 bits
    bases = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 44] 
    num_bases_to_generate = rounds - len(bases)
    for _ in range(num_bases_to_generate):
        # Append random bases into base
        random_base = random.randint(2, n -2)
        if random_base not in bases:
            bases.append(random_base)
    
    # test for primes
    for base in bases:
        x = pow(base, d, n)
        for i in range(s):
            y = pow(x, 2, n)
            if y == 1 and x != 1 and x != n-1:
                return False
            x = y
        if y != 1:
            return False
    
    return True

def get_prime_factor(n: int) -> dict[int, int]:
    temp = n
    n_factors: dict[int, int] = {}
    while (temp > 1):
        # Check if it's prime 
        if is_prime(temp):
            n_factors[temp] = 1
            break
        # Find a factor
        while (True):
            if temp < (2 << 32):
                factor = smallest_prime_factor(temp)
            else:
                factor = polland_factor(temp)
            if not factor:
                pprint("Retrying the function...!")
            else:
                break
        # Factor that number out of temp
        exp = 0
        while(temp > 1 and temp % factor == 0):
            temp //= factor
            exp += 1
        # Recursive factorization if the factor is not prime
        if is_prime(factor):
            n_factors[factor] = exp
        else:
            factor_primes = get_prime_factor(factor)
            # Update primes inside factors accordingly
            # to the number of factor's exponent
            if exp > 1:
                for prime in factor_primes.keys():
                    factor_primes[prime] *= exp 
            n_factors.update(factor_primes)
    return n_factors

def smallest_prime_factor(n: int):
    if n % 2 == 0:
        return 2
    if n % 3 == 0:
        return 3
    d = 5
    while d * d <= n:
        if n % d == 0:
            return d
        if (n % (d + 2)) == 0:
            return d + 2
        d += 6
    return d

def polland_factor(n :int):    
    last_x = x = 2
    last_y = y = x
    d = 1
    
    b = random.randint(1, n-2)
    pprint("Polland Rho algo!")
    pprint(f"{n=}\t{b=}")
    def g(a: int):
        return (a * a + b) % n
    
    steps = 100
    max_outer_loop = 1000000
    found_d = False
    for loop in range(max_outer_loop):
        if (d != 1):
            found_d = True
            break
        product = 1
        steps_failed = False
        for i in range(steps):
            x = g(x) # turtle
            y = g(g(y)) # hare
            if x == y:
                # Algo failed, fall back to step = 1
                pprint(f"Multiple step failed at {i}! Reverting to step 1!")
                steps_failed = True
                break
            product = (product * abs(x - y)) % n
        
        # Termination condition
        if steps_failed and steps == 1:
            pprint("x and y converge!")
            return None
        
        # Set step to 1
        if steps != 1 and (steps_failed or loop >= max_outer_loop // 2):
            print("Setting steps to 1...")
            steps = 1
            x = last_x
            y = last_y
            continue

        last_x = x
        last_y = y
        
        d = gcd_euclid(product, n)
    
    if d == n:
        pprint("d meets n!")
        return None
    if not found_d:
        pprint("Maximum global step reached!")
        return None
    pprint(f"Found {d=}")
    return d

In [1]:
a = 1
b = 7
p = 81663996540811672901764249733343363790991183353803305739092974199965546219729
G = (14023374736200111073976017545954000619736741127496973904317708826835398305431, 23173384182409394365116200040829680541979866476670477159886520495530923549144)
P = (45277951688968912485631557795066607843633896482130484276521452596515645125170, 33416418291776817124002088109454937261688060362650609033690500364258401702752)
ciphertext = '44af53c95092c86c04b67358aad3911282347862fec02f8943ea2eb5297780a7098faef27b2d2dbab7cf29bec5e32adcc7be6f4b57370aa2b6f6d1eafc5c3f3a07db1162d00b0037b757450b6fd405e0'
iv = '29d6bba244e66a562969a6dae8e61449'

E = EllipticCurve(GF(p), [a, b])
G = E(G)

n = G.order()

In [2]:
print(get_prime_factor(p-1))

NameError: name 'get_prime_factor' is not defined

In [3]:
print(n)

27221332180270557633921416577781121263650323017173283011972894350705530180203


In [4]:
block_size = 32
cipherhex = ciphertext
cipher_blocks = [cipherhex[i:i+block_size] for i in range(0, len(ciphertext), block_size)]
print(cipher_blocks)

['44af53c95092c86c04b67358aad39112', '82347862fec02f8943ea2eb5297780a7', '098faef27b2d2dbab7cf29bec5e32adc', 'c7be6f4b57370aa2b6f6d1eafc5c3f3a', '07db1162d00b0037b757450b6fd405e0']


In [11]:
F = factor(20)
print(F)

2^2 * 5
